# Distributed Hyper-parameter Optimization on Cori notebook

In this notebook we will develop a Cori example for HPO.

Let's develop the notebook in stages:
1. Make it so we can invoke python and run training of a single job from the notebook on the login node.
2. Launch the single job on the batch system using the SlurmJob API.
3. Develop HPO logic
4. Demonstrate distributed HPO with multi-node training

In [1]:
cd ..

/global/u2/s/sfarrell/WorkAreas/jupyter-dl/pytorch-examples


In [2]:
import os
import yaml
import numpy as np

In [3]:
from utils.slurm_helpers import SlurmJob

## Environment setup

Let's start by editing our environment directly.
I might also prefer to do this in the kernel json file.

In [12]:
os.environ['PATH'] = '/usr/common/software/pytorch/v0.4.1/bin:' + os.environ['PATH']
os.environ['LD_LIBRARY_PATH'] = '/usr/common/software/pytorch/v0.4.1/lib:' + os.environ['LD_LIBRARY_PATH']
os.environ['MPICH_MAX_THREAD_SAFETY'] = 'multiple'
os.environ['KMP_AFFINITY'] = 'granularity=fine,compact,1,0'
os.environ['KMP_BLOCKTIME'] = '1'

In [14]:
hpo_dir = os.path.expandvars('$SCRATCH/pytorch-examples/mnist-hpo')
os.makedirs(hpo_dir, exist_ok=True)

## Useful functions

In [13]:
def build_config(output_dir, conv_sizes, dense_sizes,
                 learning_rate=0.001, optimizer='Adam',
                 batch_size=64, n_epochs=1):
    data_config = dict(name='mnist', data_path='$SCRATCH/pytorch-mnist/data')
    experiment_config = dict(name='basic', output_dir=output_dir)
    model_config = dict(
        model_type='cnn_classifier',
        input_shape=[1, 28, 28], n_classes=10,
        conv_sizes=conv_sizes, dense_sizes=dense_sizes,
        optimizer=optimizer, learning_rate=learning_rate
    )
    train_config = dict(batch_size=batch_size, n_epochs=n_epochs)
    return dict(data_config=data_config, experiment_config=experiment_config,
                model_config=model_config, train_config=train_config)

def get_val_acc(config):
    output_dir = os.path.expandvars(config['experiment_config']['output_dir'])
    summaries = np.load(os.path.join(output_dir, 'summaries.npz'))
    return summaries['valid_acc'].max()

## Run training locally

In [ ]:
config = build_config(output_dir=os.path.join(hpo_dir, 'output'),
                      conv_sizes=[8, 16, 32], dense_sizes=[])

In [ ]:
# Serialize the configuration to a temporary file
config_file = os.path.join(hpo_dir, 'test.yaml')
with open(config_file, 'w') as f:
    yaml.dump(config, f)

In [ ]:
!python ./main.py $config_file

In [ ]:
print('Validation set accuracy:', get_val_acc(config))

## Run training on batch system

In [ ]:
# Job configuration
job_config = dict(
    node_type='haswell',
    n_nodes=1,
    qos='interactive',
    time=30,
)

In [ ]:
job = SlurmJob(**job_config)

In [ ]:
out, err = job.submit_task('python ./main.py %s' % config_file).communicate()

In [ ]:
# Our python logging currently goes to stderr
print(err.decode())

In [ ]:
print('Validation set accuracy:', get_val_acc(config))

In [ ]:
# End the allocation
del job

## Define HP sets

In [18]:
n_hpo_trials = 8

# Hyper-parameters for model config
c1 = np.random.choice([4, 8, 16], size=n_hpo_trials)
c2 = np.random.choice([4, 8, 16], size=n_hpo_trials)
c3 = np.random.choice([8, 16, 32], size=n_hpo_trials)
lr = np.random.choice([0.0001, 0.001, 0.01], size=n_hpo_trials)
conv_sizes = np.stack([c1, c2, c3], axis=1)

# Training config
batch_size = 64
n_epochs = 4

In [19]:
# Build the configurations for the HPO tasks
configs = [
    build_config(output_dir=os.path.join(hpo_dir, 'hp_%i' % i),
                 conv_sizes=conv_sizes[i], dense_sizes=[], learning_rate=lr[i],
                 batch_size=batch_size, n_epochs=n_epochs)
    for i in range(n_hpo_trials)
]

## Launch HP tasks to batch system

In [23]:
# Job configuration
job_config = dict(
    node_type='haswell',
    n_nodes=16,
    qos='interactive',
    time='2:00:00',
)

# Fix thread settings for remote job
os.environ['OMP_NUM_THREADS'] = '32'

In [25]:
# Start the job
job = SlurmJob(**job_config)

Launched in background. Redirecting stdin to /dev/null
salloc: Pending job allocation 15798824
salloc: job 15798824 queued and waiting for resources
salloc: job 15798824 has been allocated resources
salloc: Granted job allocation 15798824
salloc: Waiting for resource configuration
salloc: Nodes nid000[83-98] are ready for job



In [30]:
# Multi-node training configuration
n_nodes_per_task = 4

In [46]:
results = []
for i, config in enumerate(configs):
    output_dir = config['experiment_config']['output_dir']
    
    # Write the configuration to file
    os.makedirs(output_dir, exist_ok=True)
    config_file = os.path.join(output_dir, 'config.yaml')
    with open(config_file, 'w') as f:
        yaml.dump(config, f)
    
    # Submit the task
    results.append(job.submit_task('python ./main.py -d %s' % config_file, n_nodes=n_nodes_per_task))

In [47]:
jobid = job.jobid

In [49]:
!sacct -j $jobid

       JobID    JobName  Partition    Account  AllocCPUS      State ExitCode 
------------ ---------- ---------- ---------- ---------- ---------- -------- 
15798824     allocation interacti+    dasrepo       1024    RUNNING      0:0 
15798824.ex+     extern               dasrepo       1024    RUNNING      0:0 
15798824.0       python               dasrepo          4  COMPLETED      0:0 
15798824.1       python               dasrepo          4  COMPLETED      0:0 
15798824.2       python               dasrepo          4  COMPLETED      0:0 
15798824.3       python               dasrepo          4  COMPLETED      0:0 
15798824.4       python               dasrepo          4  COMPLETED      0:0 
15798824.5       python               dasrepo          4  COMPLETED      0:0 
15798824.6       python               dasrepo          4  COMPLETED      0:0 
15798824.7       python               dasrepo          4  COMPLETED      0:0 
15798824.8       python               dasrepo          4  COMPLE

In [50]:
# Wait and gather all the results
outputs = [r.communicate() for r in results]

In [51]:
# Show the full output from one job
print(outputs[0][1].decode())

Launched in background. Redirecting stdin to /dev/null
srun: Job 15798824 step creation temporarily disabled, retrying
srun: Step created for job 15798824
2018-10-16 14:57:57,706 INFO Initializing
2018-10-16 14:57:57,714 INFO Initializing
2018-10-16 14:57:57,737 INFO Initializing
2018-10-16 14:58:02,653 INFO Initializing
2018-10-16 14:58:02,699 INFO MPI rank 0
2018-10-16 14:58:02,701 INFO MPI rank 2
2018-10-16 14:58:02,701 INFO MPI rank 3
2018-10-16 14:58:02,700 INFO MPI rank 1
2018-10-16 14:58:02,717 INFO Configuration: {'data_config': {'data_path': '$SCRATCH/pytorch-mnist/data', 'name': 'mnist'}, 'experiment_config': {'name': 'basic', 'output_dir': '/global/cscratch1/sd/sfarrell/pytorch-examples/mnist-hpo/hp_0'}, 'model_config': {'conv_sizes': array([ 8, 16,  8]), 'dense_sizes': [], 'input_shape': [1, 28, 28], 'learning_rate': 0.001, 'model_type': 'cnn_classifier', 'n_classes': 10, 'optimizer': 'Adam'}, 'train_config': {'batch_size': 64, 'n_epochs': 4}}
2018-10-16 14:58:03,192 INFO L

In [59]:
# Gather the validation set accuracies
val_accs = np.array([get_val_acc(config) for config in configs])
print('Validation set accuracies:', val_accs)

Validation set accuracies: [0.9742 0.976  0.9841 0.9747 0.9778 0.8979 0.9776 0.8976]


In [58]:
# Best model configuration
configs[val_accs.argmax()]

{'data_config': {'name': 'mnist', 'data_path': '$SCRATCH/pytorch-mnist/data'},
 'experiment_config': {'name': 'basic',
  'output_dir': '/global/cscratch1/sd/sfarrell/pytorch-examples/mnist-hpo/hp_2'},
 'model_config': {'model_type': 'cnn_classifier',
  'input_shape': [1, 28, 28],
  'n_classes': 10,
  'conv_sizes': array([ 8,  4, 32]),
  'dense_sizes': [],
  'optimizer': 'Adam',
  'learning_rate': 0.01},
 'train_config': {'batch_size': 64, 'n_epochs': 4}}